In [12]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import PromptTemplate

In [10]:
# Ollamaモデルの設定
model = ChatOllama(
    baseUrl="http://localhost:11434",  # OllamaサーバーのURL
    model="hf.co/lmstudio-community/Llama-3.3-70B-Instruct-GGUF:Q4_K_M",  # 使用するモデル名
    temperature=0.5 # 応答の多様性
)

# プロンプトテンプレートの設定
messages = [
    SystemMessage("You are helpful assistant."),
    HumanMessage("こんにちは！元気？"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

こんにちは！私はAIなので、感情を持っていませんが、いつでもお手伝いできる状態です。今日はどうしましたか？何かお助けできることがありますか？

In [15]:
prompt = PromptTemplate.from_template(
    """以下の料理のレシピを考えてください。
    
    料理名: {dish}"""
)

prompt_value = prompt.invoke({"dish": "チャーハン"})

for chunk in model.stream(prompt_value):
    print(chunk.content, end="", flush=True)

チャーハンのレシピです。

###材料：

-  ご飯：2杯
-  卵：2個
-  にんじん：1/2本
-  玉ねぎ：1/4個
-  エビ（冷凍）：100g
-  塩：適量
-  こしょう：適量
-  油：大さじ2

### 作り方：

1.  玉ねぎとにんじんをみじん切りにします。
2.  フライパンに油を入れ、火を中火で加えます。熱くなったら、にんじんと玉ねぎを加えて炒めます。
3.  にんじんや玉ねぎが柔らかくなったら、エビを加えてさらに炒めます。
4.  次に、ご飯をフライパンに加え、全体をよく混ぜます。
5.  卵を別のボウルで溶き、少し塩とこしょうを加えて混ぜます。卵液をフライパンに流し入れ、全体を混ぜながら炒めます。
6.  最後に、味を見て必要に応じて塩やこしょうを調節します。
7.  ご飯が均等に火が通ったら、完成です。

In [21]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to cook the dish")

In [22]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to cook the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [29]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Think of a recipe for a dish entered by the user in Japanese.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [30]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "チャーハン"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
Think of a recipe for a dish entered by the user in Japanese.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to cook the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
チャーハン


In [31]:
model = ChatOllama(
    baseUrl="http://localhost:11434",  # OllamaサーバーのURL
    model="hf.co/lmstudio-community/Llama-3.3-70B-Instruct-GGUF:Q4_K_M",  # 使用するモデル名
    temperature=0 # 応答の多様性
)

ai_messages = model.invoke(prompt_value)
print(ai_messages.content)

Here is a recipe for チャーハン (Chahan, Japanese Fried Rice) in JSON format:

```
{
  "ingredients": [
    "cooked rice",
    "vegetable oil",
    "onion",
    "garlic",
    "ginger",
    "scallions",
    "soy sauce",
    "salt",
    "pepper"
  ],
  "steps": [
    "Heat the vegetable oil in a large skillet or wok over medium-high heat.",
    "Add the diced onion, garlic, and ginger and stir-fry until they are softened.",
    "Add the cooked rice to the skillet or wok and stir-fry for about 5 minutes, breaking up any clumps with a spatula.",
    "Add the soy sauce, salt, and pepper to the rice mixture and stir-fry for another minute.",
    "Taste and adjust the seasoning as needed.",
    "Transfer the Chahan to a serving platter and garnish with scallions."
  ]
}
```

Note: You can customize the ingredients and steps to your liking, and also add other ingredients such as cooked meat, seafood, or vegetables to make the dish more flavorful.


In [32]:
recipe = output_parser.invoke(ai_messages)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['cooked rice', 'vegetable oil', 'onion', 'garlic', 'ginger', 'scallions', 'soy sauce', 'salt', 'pepper'] steps=['Heat the vegetable oil in a large skillet or wok over medium-high heat.', 'Add the diced onion, garlic, and ginger and stir-fry until they are softened.', 'Add the cooked rice to the skillet or wok and stir-fry for about 5 minutes, breaking up any clumps with a spatula.', 'Add the soy sauce, salt, and pepper to the rice mixture and stir-fry for another minute.', 'Taste and adjust the seasoning as needed.', 'Transfer the Chahan to a serving platter and garnish with scallions.']


In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Think of a recipe for a dish entered by the user in Japanese."),
        ("human", "{dish}"),
    ]
)

model = ChatOllama(
    baseUrl="http://localhost:11434",  # OllamaサーバーのURL
    model="hf.co/lmstudio-community/Llama-3.3-70B-Instruct-GGUF:Q4_K_M",  # 使用するモデル名
    temperature=0 # 応答の多様性
)

In [35]:
chain = prompt | model

In [36]:
ai_messages = chain.invoke({"dish": "チャーハン"})
print(ai_messages.content)

A classic Japanese dish! 

Here's a simple recipe for チャーハン (Chāhan), also known as Fried Rice:

Ingredients:

* 2 cups cooked Japanese rice (preferably day-old rice)
* 1 tablespoon vegetable oil
* 1 small onion, diced
* 2 cloves garlic, minced
* 1 cup mixed vegetables (e.g., peas, carrots, corn)
* 2 eggs, beaten
* 1 teaspoon soy sauce
* 1 teaspoon oyster sauce (optional)
* Salt and pepper to taste
* Scallions, chopped (for garnish)

Instructions:

1. Heat the oil in a large skillet or wok over medium-high heat.
2. Add the diced onion and cook until translucent.
3. Add the minced garlic and cook for another minute.
4. Add the mixed vegetables and cook until they're tender-crisp.
5. Push the vegetables to one side of the pan. Crack in the beaten eggs and scramble them until cooked through. Then, mix the eggs with the vegetables.
6. Add the cooked rice to the pan, breaking up any clumps with a spatula. Stir-fry the rice with the vegetables and eggs for about 5 minutes, or until everythin

In [37]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "チャーハン"})
print(output)

A classic Japanese dish! 

Here's a simple recipe for チャーハン (Chāhan), also known as Fried Rice:

Ingredients:

* 2 cups cooked Japanese rice (preferably day-old rice)
* 1 tablespoon vegetable oil
* 1 small onion, diced
* 2 cloves garlic, minced
* 1 cup mixed vegetables (e.g., peas, carrots, corn)
* 2 eggs, beaten
* 1 teaspoon soy sauce
* 1 teaspoon oyster sauce (optional)
* Salt and pepper to taste
* Scallions, chopped (for garnish)

Instructions:

1. Heat the oil in a large skillet or wok over medium-high heat.
2. Add the diced onion and cook until translucent.
3. Add the minced garlic and cook for another minute.
4. Add the mixed vegetables and cook until they're tender-crisp.
5. Push the vegetables to one side of the pan. Crack in the beaten eggs and scramble them until cooked through. Then, mix the eggs with the vegetables.
6. Add the cooked rice to the pan, breaking up any clumps with a spatula. Stir-fry the rice with the vegetables and eggs for about 5 minutes, or until everythin

Subject: =?UTF-8?B?6YeN5aSn44Gq44K744Kt44Ol44Oq44OG44Kj6YCa55+l?=
From: Google <no-reply@accounts.google.com>
Content: None
Subject: =?iso-2022-jp?B?UmU6IBskQiVsJXMlPyVrSipJSiRLNFgkNyReJDckRhsoQg==?=
From: =?iso-2022-jp?B?GyRCQ3Y8bSEhMyEbKEI=?= <s2210502@s.tsukuba.ac.jp>
Content: None
Subject: =?UTF-8?B?44Os44Oz44K/44Or54mp5ZOB44Gr6Zai44GX44G+44GX44Gm?=
From: =?UTF-8?B?562R5rOi5aSn5a2m5a2m5ZyS56Wt5a6f6KGM5aeU5ZOh5Lya?= <project50th@sohosai.com>
Content: None
Subject: 
 =?iso-2022-jp?B?UmU6IBskQiFaM1g8QjBRIVs0R0hEJE4kNEpWNVEkSyREJCQkRhsoQg==?=
From: =?iso-2022-jp?B?GyRCMGxMbjEpMCFGYBsoQg==?= <s2210773@u.tsukuba.ac.jp>
Content: None
Subject: 
 =?iso-2022-jp?B?UmU6IBskQiFaM1g8QjBRIVs0R0hEJE4kNEpWNVEkSyREJCQkRhsoQg==?=
From: =?iso-2022-jp?B?GyRCQWoxOiEhSydNVRsoQg==?= <s2210218@s.tsukuba.ac.jp>
Content: None
Subject: =?UTF-8?B?44CQ5a2m5a6f5aeU44CR55yL5p2/44Gu44GU6L+U5Y2044Gr44Gk44GE44Gm?=
From: =?UTF-8?B?562R5rOi5aSn5a2m5a2m5ZyS56Wt5a6f6KGM5aeU5ZOh5Lya?= <project50th@sohosai.com>
Content: N